# Настройка среды

In [1]:
import os
import sys
sys.path.append(os.path.abspath("../.."))

In [2]:
from belashovplot import TiledPlot
from utilities import *
from utilities.filters import Gaussian, Window
from elements.abstracts import AbstractModulator
from elements.modulators import Lens, PhaseModulator, AmplitudeModulator
from elements.propagators import FurrierPropagation, ConvolutionalPropagation
from elements.composition import CompositeModel, HybridModel
from elements.wrappers import CudaMemoryChunker, Incoherent
from elements.detectors import ClassificationDetectors, MatrixDetectors
from elements.simple import AdjustSize
from parameters import FigureWidthHeight, FontLibrary
from tqdm import tqdm
from math import sin, sqrt
import torch
from torch.profiler import profile
import numpy
import timm
import pandas
from copy import deepcopy
from utilities.training import train

In [3]:
FontLibrary.Fonts.PlotTitle.FontSize = 16

FontLibrary.Fonts.DescriptionLeft.FontSize = 12
FontLibrary.Fonts.DescriptionLeft.FontWeight = 'bold'
FontLibrary.Fonts.DescriptionLeft.FontStyle = 'italic'

FontLibrary.Fonts.DescriptionBottom.FontSize = 10
FontLibrary.Fonts.DescriptionBottom.FontWeight = 'bold'
FontLibrary.Fonts.DescriptionBottom.FontStyle = 'italic'

FontLibrary.Fonts.DescriptionTop.FontSize = 10
FontLibrary.Fonts.DescriptionTop.FontWeight = 'bold'
FontLibrary.Fonts.DescriptionTop.FontStyle = 'italic'

FontLibrary.Fonts.ColumnDescriptionTop.FontSize = 9
FontLibrary.Fonts.ColumnDescriptionBottom.FontSize = 9
FontLibrary.Fonts.RowDescriptionLeft.FontSize = 9
FontLibrary.Fonts.RowDescriptionRight.FontSize = 9
FontLibrary.Fonts.AxisX.FontSize = 8
FontLibrary.Fonts.AxisY.FontSize = 8
FigureWidthHeight = (6.69291, 10.1181-1.0)

In [4]:
device = torch.device(7)
print('Используемый девайс:', torch.cuda.get_device_name(device) if torch.cuda.is_available() else 'ЦП')

Используемый девайс: NVIDIA A100-SXM4-80GB


# Определение параметров модели

In [5]:
def resnet18(classes:int=10):
    return timm.create_model('resnet18', pretrained=False, in_chans=1, num_classes=classes).to(device)

In [6]:
def combined_loss(cross_entropy_to_mse_proportion:float=1.0):
    def loss_function(outputs, targets):
        CELoss = torch.nn.functional.cross_entropy(outputs, targets)
        MSELoss = torch.nn.functional.mse_loss(torch.nn.functional.softmax(outputs, dim=1), torch.nn.functional.one_hot(targets, num_classes=10).float())
        loss = cross_entropy_to_mse_proportion*CELoss + (1.0-cross_entropy_to_mse_proportion)*MSELoss
        return loss
    return loss_function

In [7]:
# Предпочтительные параметры
size = 30.0E-6
near_N = 400 #2004 #1336
near_length = 6.0E-3
wavelength = 500.0E-9
detectors_amount = 24
masks_amount = 10
distance = 0.05

# Параметры обучения
batch_size = 32
learning_rate = 0.009854
loss_function_proportion = 0.609798
optimizer_type_name = 'Adam'
optimizer_types_list = {'Adam':torch.optim.Adam, 'SGD':torch.optim.SGD, 'RMSprop':torch.optim.RMSprop, 'Adagrad':torch.optim.Adagrad}

# Параметры когерентности
spatial_coherence = 100.0E-6
time_coherence = 10.0E-9
time_relaxation = 1.0E-6
mean_samples = 10

# Вычисляемые параметры
pixels = upper_integer(near_length/size)
length = pixels * size
cppp = int(near_N * size / length)
N = upper_integer(length*cppp / size) 
detector_size = length / 60

print(f"Длинна волны:                             {engineering(wavelength, 'м')}")
print(f"Количество вычислительных пикселей:       {N}")
print(f"Количество пикселей маски:                {pixels}")
print(f"Размер оптических элементов:              {engineering(length, 'м')}")
print(f"Размер пикселя маски:                     {engineering(length/pixels, 'м')}")
print(f"Расстояние между слоями:                  {engineering(distance, 'м')}")

print(f"Временная когерентность:                  {engineering(time_coherence, 'с')}")
print(f"Время релаксации:                         {engineering(time_relaxation, 'c')}")
print(f"Пространственная когерентность:           {engineering(spatial_coherence, 'м')}")
print(f"Количество усреднений:                    {mean_samples}")

print(f"Размер детекторов:                        {engineering(detector_size, 'м')}")
print(f"Количество детекторов:                    {detectors_amount} на {detectors_amount}")

print(f"Пропорция CE к MSE лосс:                  {loss_function_proportion}")
print(f"Размер батча:                             {batch_size}")
print(f"Тип оптимизатора:                         {optimizer_type_name}")

Длинна волны:                             500.0 нм
Количество вычислительных пикселей:       400
Количество пикселей маски:                200
Размер оптических элементов:              6.0 мм
Размер пикселя маски:                     30.0 мкм
Расстояние между слоями:                  50.0 мм
Временная когерентность:                  10.0 нс
Время релаксации:                         1.0 мкc
Пространственная когерентность:           100.0 мкм
Количество усреднений:                    10
Размер детекторов:                        100.0 мкм
Количество детекторов:                    24 на 24
Пропорция CE к MSE лосс:                  0.609798
Размер батча:                             32
Тип оптимизатора:                         Adam


# Инициализация моделей

In [8]:
# Модуль некогерентности
incoherent = Incoherent(spatial_coherence, time_coherence, time_relaxation, mean_samples, N, length)
incoherent_encoder, incoherent_decoder = incoherent.pair()
incoherent_encoder, incoherent_decoder = incoherent_encoder.to(device), incoherent_decoder.to(device)

# Модули детекторов
spectral_filter = Window(centers=wavelength, sizes=300.0E-9)
detectors_filter = Gaussian((detector_size, detector_size), (0,0))
detectors_incoherent = MatrixDetectors(N, length, wavelength, detectors_amount, detectors_filter, spectral_filter).to(device)
detectors_coherent = deepcopy(detectors_incoherent)

# Электронные модели
electronic_incoherent = resnet18()
electronic_coherent = deepcopy(electronic_incoherent)

# Оптические модели
adjuster = AdjustSize(N, N)
propagation = FurrierPropagation(N, length, wavelength, 1.0, 0.0, distance, 0.4)
phase_modulators = [PhaseModulator(N, length, pixels) for i in range(masks_amount)]
amplitude_modulators = [AmplitudeModulator(N, length, pixels) for i in range(masks_amount)]
elements = [phase_modulators[0], amplitude_modulators[0]]
for phase_modulator, amplitude_modulator in zip(phase_modulators[1:], amplitude_modulators[1:]):
    elements.append(propagation)
    elements.append(phase_modulator)
    elements.append(amplitude_modulator)
elements.append(propagation)
optical_incoherent = CompositeModel(adjuster, incoherent_encoder, *deepcopy(elements), incoherent_decoder).to(device)

# Гибридные модели
hybrid_model_incoherent = HybridModel(optical_incoherent, detectors_incoherent, electronic_incoherent).to(device)

# Распределённые модели
model_incoherent = hybrid_model_incoherent

# Набор данных
dataset = Dataset('CIFAR10', batch_size, None, None, torch.complex64, threads=1, preload=10)
dataset.load.train()
dataset.load.test()

# Тип оптимизатора и лосс-функция
loss_function = combined_loss(loss_function_proportion)
optimizer_type = optimizer_types_list[optimizer_type_name]
optimizer = optimizer_type(model_incoherent.parameters())

# Тренировка модели

In [9]:
class IterationBreaker:
    def __init__(self, loader:torch.utils.data.DataLoader, break_after:int):
        self.loader = loader
        self.break_after = break_after
    def __iter__(self):
        self.counter = 0
        self.iterator = iter(self.loader)
        return self
    def __next__(self):
        image, label = next(self.iterator)
        if self.counter >= self.break_after:
            raise StopIteration
        self.counter += 1
        return image, label
    def __len__(self):
        return min(self.break_after, len(self.loader))

In [10]:
from time import time

loss_function = torch.nn.CrossEntropyLoss()
model = model_incoherent
with profile(activities=[torch.profiler.ProfilerActivity.CPU, torch.profiler.ProfilerActivity.CUDA], record_shapes=True, profile_memory=True, with_stack=True, with_flops=True, with_modules=True, on_trace_ready=torch.profiler.tensorboard_trace_handler(f"profile.log")) as prof:
    model.train()
    history = numpy.zeros((len(dataset.train)))

    time_start = time()
    running_loss = 0
    running_loss_proportion = 0.2
    regression = 0

    iterator = tqdm(IterationBreaker(dataset.train, 3))

    for i, (images, labels) in enumerate(iterator):
        labels = labels.to(device, non_blocking=True)
        images = images.to(device, non_blocking=True)

        results = model.forward(images)

        optimizer.zero_grad()
        loss = loss_function(results, labels)
        loss.backward()
        optimizer.step()

        history[i] = loss.item()
        if running_loss == 0:   running_loss = loss.item()
        else:                   running_loss = (1.0 - running_loss_proportion)*running_loss + running_loss_proportion*loss.item()

        if time() - time_start >= 10.0 and i >= 3:
            time_start = time()
            history_slice = history[:i+1]
            iteration_slice = numpy.arange(0, i+1)
            k, b = numpy.polyfit(iteration_slice, history_slice, 1)
            regression = k * 1000
        iterator.set_description(f"RLoss: {running_loss}, RPI1000: {regression}")
    model.eval()

STAGE:2024-07-11 10:42:49 1257509:1257509 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
RLoss: 2.4146196937561037, RPI1000: 0: 100%|██████████████████████████████████████████████| 3/3 [00:04<00:00,  1.38s/it]
STAGE:2024-07-11 10:42:55 1257509:1257509 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2024-07-11 10:42:55 1257509:1257509 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


In [14]:
!pip install tensorboard
!tensorboard --logdir=profile.log

Defaulting to user installation because normal site-packages is not writeable
Traceback (most recent call last):
  File "/home/encoder/.local/bin/tensorboard", line 5, in <module>
    from tensorboard.main import run_main
  File "/home/encoder/.local/lib/python3.10/site-packages/tensorboard/main.py", line 27, in <module>
    from tensorboard import default
  File "/home/encoder/.local/lib/python3.10/site-packages/tensorboard/default.py", line 39, in <module>
    from tensorboard.plugins.hparams import hparams_plugin
  File "/home/encoder/.local/lib/python3.10/site-packages/tensorboard/plugins/hparams/hparams_plugin.py", line 30, in <module>
    from tensorboard.plugins.hparams import backend_context
  File "/home/encoder/.local/lib/python3.10/site-packages/tensorboard/plugins/hparams/backend_context.py", line 26, in <module>
    from tensorboard.plugins.hparams import metadata
  File "/home/encoder/.local/lib/python3.10/site-packages/tensorboard/plugins/hparams/metadata.py", line 32, i